# core

> Fill in a module description here


In [115]:
# | default_exp logger

In [116]:
# | hide
import nbdev

nbdev.nbdev_export()

In [117]:
from typing import List, Dict, Any, Optional, Union
from pydantic import BaseModel, Field


from lovely_prompts_server.schemas import ChatPromptBase, ChatResponseBase, ChatPrompt, ChatResponse

In [118]:
# class PromptBase(BaseModel):
#     title: str = Field(None, example="The encounter")
#     prompt: Union[List[Dict], str] = Field(None, example={"messages": [{"role": "user", "content": "Hello there"}]})
#     comment: str = Field(None, example="This is a comment")
#     project: str = Field(None, example="my-awesome-project")

#     class Config:
#         orm_mode = True


# class ResponseBase(BaseModel):
#     prompt_id: int = Field(None, example=1)
#     title: str = Field(None, example="The response")
#     response: Dict = Field(None, example="General Kenobi!!")
#     comment: str = Field(None, example="This is a response comment")
#     tok_in: int = Field(None, example=10)
#     tok_out: int = Field(None, example=20)
#     tok_max: int = Field(None, example=8000)
#     meta: Dict = Field(None)
#     model: str = Field(None, example="gpt-3.5-turbo")
#     temperature: float = Field(None, example=0.7)
#     provider: str = Field(None, example="openai")


# class Response(ResponseBase):
#     id: int
#     # prompt: PromptBase = Field(..., alias="prompt")


# class Prompt(PromptBase):
#     id: int
#     responses: List[ResponseBase] = Field([], alias="responses")

In [119]:
import requests
import asyncio

In [120]:
class Logger:
    def __init__(
        self,
        run_server=False,
        url_base: str = None,
        api_key: Optional[str] = None,
        project: Optional[str] = None,
        provider: Optional[str] = None,
    ):
        self.url_base = url_base or "http://localhost:8000"
        self.api_key = api_key
        self.project = project
        self.provider = provider

        if run_server:
            print("Starting server...")
            import uvicorn
            from lovely_prompts_server import app

            try:
                loop = asyncio.get_running_loop()
                config = uvicorn.Config(app, host="localhost", port=8000)
                server = uvicorn.Server(config)
                loop.create_task(server.serve())
            except Exception as e:
                print(e)

                # uvicorn.run(app, host="localhost", port=8000, )

    def log_chat_prompt(
        self,
        messages: List[Dict],
        title: Optional[str] = None,
        comment: Optional[str] = None,
        project: Optional[str] = None,
    ) -> int:
        prompt_data = ChatPromptBase(
            title=title, messages=messages, comment=comment, project=project if project else self.project
        )
        response = requests.post(f"{self.url_base}/prompts/", json=prompt_data.dict())
        if response.status_code != 200:
            raise Exception(f"Failed to log prompt, status code: {response.status_code}")
        return ChatPrompt(**response.json()).id

    def log_chat_response(
        self,
        prompt_id: Union[str, int],
        response: Dict,
        title: Optional[str] = None,
        comment: Optional[str] = None,
        tok_in: Optional[int] = None,
        tok_out: Optional[int] = None,
        tok_max: Optional[int] = None,
        meta: Optional[Dict] = None,
        model: Optional[str] = None,
        temperature: Optional[float] = None,
    ) -> int:
        response_data = ChatResponseBase(
            prompt_id=prompt_id,
            title=title,
            response=response,
            comment=comment,
            tok_in=tok_in,
            tok_out=tok_out,
            tok_max=tok_max,
            meta=meta,
            model=model,
            provider=self.provider,
            temperature=temperature,
        )
        response = requests.post(f"{self.url_base}/responses/", json=response_data.dict())
        if response.status_code != 200:
            raise Exception(f"Failed to log response, status code: {response.status_code}")
        return ChatResponse(**response.json()).id

In [121]:
from dotenv import load_dotenv

load_dotenv()

True

In [122]:
import openai

# chat = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo", temperature=0.9, max_tokens=150, messages=[{"role": "user", "content": "Hello there"}]
# )

In [123]:
logger = Logger(run_server=False, project="Test project", provider="openai")

In [124]:
messages = [{"role": "user", "content": "Hello there"}]

prompt_id = logger.log_chat_prompt(messages, comment="This is a comment", title="The encounter")

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", temperature=0.9, max_tokens=150, messages=[{"role": "user", "content": "Hello there"}]
)

# "prompt_tokens": 9,
# "completion_tokens": 9,
# "total_tokens": 18

logger.log_chat_response(
    prompt_id,
    response.choices[0].message,
    comment="This is a comment",
    title="The response",
    model=response.model,
    temperature=0.7,
    tok_in=response.usage.prompt_tokens,
    tok_out=response.usage.completion_tokens,
    tok_max=8000,
)

9

In [125]:
import asyncio


def inside_async_loop():
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False


print(inside_async_loop())  # False if not inside a running event loop

True
